In [2]:
import datetime
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
from datetime import datetime as dt
import numba as nb
from collections import defaultdict 
from tqdm.notebook import tqdm

In [3]:
if not mt5.initialize():
    print(mt5.last_error())
    mt5.shutdown()
else:
    authorized=mt5.login(18676, password="aTOirs42", server="GoMarkets-Demo")
    print(mt5.last_error())

(1, 'Success')


In [4]:
def getData(numBefore, timePeriod, signal, timeDict):
    rates = mt5.copy_rates_from_pos(signal, timeDict[timePeriod], 0, numBefore)       
    ratesDf = pd.DataFrame(rates)
    return ratesDf

# Give closePrices as an array
def getBollBands(closePrices, period=20, numStd=2):
    # Default values for bollinger bands = 20 for period & 2 for std
    bands = {k:[] for k in ['topBand','bottomBand', 'mav']} # Set lists as defaults

    for i in range(1, len(closePrices)+1):
        if i < period:
            currDat = closePrices[0:i]
        else:
            currDat = closePrices[i-period:i]

        std = np.std(currDat)
        mean = np.mean(currDat)

        bands['topBand'].append(mean+numStd*std)
        bands['bottomBand'].append(mean-numStd*std)
        bands['mav'].append(mean)

    # Remove the first n (period) of values as they are not accurate/just used for calculation
    for key, val in bands.items():
        tempVal = val.copy()
        tempVal[:period] = [float('nan')]*period
        bands[key] = tempVal

    bollDf = pd.DataFrame.from_dict(bands)
    return bollDf

@nb.jit(fastmath=True, nopython=True)   
def calc_rsi( array, deltas, avg_gain, avg_loss, n ):

# Use Wilder smoothing method
    up   = lambda x:  x if x > 0 else 0
    down = lambda x: -x if x < 0 else 0
    i = n+1
    for d in deltas[n+1:]:
        avg_gain = ((avg_gain * (n-1)) + up(d)) / n
        avg_loss = ((avg_loss * (n-1)) + down(d)) / n
        if avg_loss != 0:
            rs = avg_gain / avg_loss
            array[i] = 100 - (100 / (1 + rs))
        else:
            array[i] = 100
        i += 1

    return array

# Default period for the RSI indicator is 14
def getRSI(array, n = 14):   

    deltas = np.append([0],np.diff(array))

    avg_gain =  np.sum(deltas[1:n+1].clip(min=0)) / n
    avg_loss = -np.sum(deltas[1:n+1].clip(max=0)) / n

    array = np.empty(deltas.shape[0])
    array.fill(np.nan)

    array = calc_rsi( array, deltas, avg_gain, avg_loss, n )
    return array

## Get the dataframe with information of rates, RSI and bollinger bands
def getAllInfo(rates):  
    boll = getBollBands(rates.close)
    rsi = getRSI(rates.close)
    
    times = rates['time'].values
        
    # First get the RSI, signal and Bollinger bands in a dataframe with respect to time
    algDf = pd.DataFrame()
    algDf['Date'] = times
    algDf = algDf.set_index('Date')
    
    # Add the RSI values
    algDf['RSI'] = list(rsi)
    
    # Add the signal values (using the close value)
    ratesDf = pd.DataFrame(rates.drop(columns = ['time', 'tick_volume', 'spread', 'real_volume']))
    ratesDf['Date'] = times
    ratesDf = ratesDf.set_index('Date')
    
    # Add the bollinger bands (can use the mav to figure out the take-profit value)
    boll.reset_index(inplace=True)
    boll['Date'] = times
    boll = boll.set_index('Date')
        
    concatDf = pd.concat([algDf, boll, ratesDf], axis=1)
    concatDf.drop(columns=['index'], inplace=True)
    
    filteredDf = concatDf[concatDf.mav > 0]
    filteredDf.reset_index(inplace=True)
    
    return filteredDf

## Bollinger band test on all GBP pairs

In [ ]:
timeDict = {'15': mt5.TIMEFRAME_M15, '30': mt5.TIMEFRAME_M30, '60': mt5.TIMEFRAME_H1}

barsAfter = [3, 5, 12]
resultDict = {}

signals = mt5.symbols_get(group="*GBP*, !*EUR*, !*JPY*")
# signals = mt5.symbols_get(group="*GBP*, *JPY*, *EUR*, *AUD*, *GBP*, *CHF*, *NZD*, *CAD*")

sigNames = [sig.name for sig in signals]
print(sigNames)

pips = 0.001

for sig in sigNames:
    print(sig)
    
    for key, val in tqdm(timeDict.items()):
        print(key)

        resultDict[key] = defaultdict(list)

        numYears = 1
        numBefore = int(60/int(key) * 24 * 365 * numYears) # Data for one year 

        print(numBefore)
        rates = getData(numBefore, key, sig, timeDict) 

        print(mt5.last_error())
        filteredDf = getAllInfo(rates)

        ## FIRST CHECK IF THERE IS ANY CORRELATION FOR A SIMPLE BREACH OF BARS
        for index, row in tqdm(filteredDf.iterrows()):
            if index > len(filteredDf)-max(barsAfter)-1:
                 continue

            # Current bar has closed outside upper band
            if row.close > row.topBand:
                for n in barsAfter:
                    nextBars = filteredDf.iloc[index+1:index+n+1]

                    if min(nextBars.low) < (row.close - pips):
                        resultDict[key][n].append(True)
                    else:
                        resultDict[key][n].append(False)

            elif row.close < row.bottomBand:

                for n in barsAfter:
                    nextBars = filteredDf.iloc[index+1:index+n+1]

                    if max(nextBars.high) > (row.close + pips):
                        resultDict[key][n].append(True)
                    else:
                        resultDict[key][n].append(False)

#     print('timeframe', 'bars_after', 'truth_values', 'percentage_true')
    for key, val in resultDict.items():
        for i, j in val.items():
            n, c = np.unique(j, return_counts=True)
            print(key, i, n, c, c[1]/sum(c)*100)
        print('\n')

## RSI test on all GBP pairs

In [5]:
timeDict = {'15': mt5.TIMEFRAME_M15, '30': mt5.TIMEFRAME_M30, '60': mt5.TIMEFRAME_H1}

threshUp = 75
threshDown = 25

barsAfter = [3, 5, 12]
resultDict = {}

# signals = mt5.symbols_get(group="*GBP*, !*EUR*, !*JPY*")
signals = mt5.symbols_get(group="*GBP*, *JPY*, *EUR*, *AUD*, *GBP*, *CHF*, *NZD*, *CAD*")

sigNames = [sig.name for sig in signals]
print(sigNames)

pips = 0.001

['EURUSD', 'GBPUSD', 'USDCHF', 'USDJPY', 'USDCAD', 'AUDUSD', 'AUDNZD', 'AUDCAD', 'AUDCHF', 'AUDJPY', 'CHFJPY', 'EURGBP', 'EURAUD', 'EURCHF', 'EURJPY', 'EURNZD', 'EURCAD', 'GBPCHF', 'GBPJPY', 'CADCHF', 'AUDHKD', 'EURDKK', 'EURHUF', 'EURMXN', 'EURNOK', 'EURPLN', 'EURSEK', 'EURTRY', 'CADJPY', 'GBPAUD', 'GBPCAD', 'GBPNZD', 'NZDCAD', 'NZDCHF', 'NZDJPY', 'NZDUSD']


In [9]:
for sig in sigNames:
    print(sig)
    
    for key, val in tqdm(timeDict.items()):
        resultDict[key] = defaultdict(list)

        numYears = 0.1
        numBefore = int(60/int(key) * 24 * 365 * numYears) # Data for one year 
        print(numBefore)
        
        rates = getData(numBefore, key, sig, timeDict) 
        filteredDf = getAllInfo(rates)
        print(mt5.last_error())       

        ## FIRST CHECK IF THERE IS ANY CORRELATION FOR A SIMPLE BREACH OF BARS
        for index, row in tqdm(filteredDf.iterrows()):
            if index > len(filteredDf)-max(barsAfter)-1:
                 continue

            # Current bar has closed outside upper band
            if row.RSI > threshUp:
    #            print(pd.to_datetime(row.Date, unit='s'))

                for n in barsAfter:
                    nextBars = filteredDf.iloc[index+1:index+n+1]

                    if min(nextBars.low) < (row.close - pips):
                        resultDict[key][n].append(True)
                    else:
                        resultDict[key][n].append(False)

            elif row.RSI < threshDown:

                for n in barsAfter:
                    nextBars = filteredDf.iloc[index+1:index+n+1]

                    if max(nextBars.high) > (row.close + pips):
                        resultDict[key][n].append(True)
                    else:
                        resultDict[key][n].append(False)
                        
    for key, val in resultDict.items():
        for i, j in val.items():
            n, c = np.unique(j, return_counts=True)
            print(key, i, n, c, c[1]/sum(c)*100)
        print('\n')

EURUSD


3504
(1, 'Success')



1752
(1, 'Success')



876
(1, 'Success')




15 3 [False  True] [99 30] 23.25581395348837
15 5 [False  True] [87 42] 32.55813953488372
15 12 [False  True] [68 61] 47.286821705426355


30 3 [False  True] [44 30] 40.54054054054054
30 5 [False  True] [36 38] 51.35135135135135
30 12 [False  True] [25 49] 66.21621621621621


60 3 [False  True] [15 24] 61.53846153846154
60 5 [False  True] [ 9 30] 76.92307692307693
60 12 [False  True] [ 5 34] 87.17948717948718


GBPUSD


...

3504
(1, 'Success')



1752
(1, 'Success')



876
(1, 'Success')




15 3 [False  True] [  6 106] 94.64285714285714
15 5 [False  True] [  4 108] 96.42857142857143
15 12 [False  True] [  3 109] 97.32142857142857


30 3 [False  True] [ 3 53] 94.64285714285714
30 5 [False  True] [ 3 53] 94.64285714285714
30 12 [False  True] [ 3 53] 94.64285714285714


60 3 [False  True] [ 1 25] 96.15384615384616
60 5 [False  True] [ 1 25] 96.15384615384616


IndexError: index 1 is out of bounds for axis 0 with size 1

## RSI + Bollinger band test on GBP pairs

In [11]:
pips = 0.0005
for sig in sigNames:
    print(sig)
    
    for key, val in tqdm(timeDict.items()):
        resultDict[key] = defaultdict(list)

        numYears = 2
        numBefore = int(60/int(key) * 24 * 365 * numYears) # Data for one year 

        rates = getData(numBefore, key, sig, timeDict) 
        filteredDf = getAllInfo(rates)

        ## FIRST CHECK IF THERE IS ANY CORRELATION FOR A SIMPLE BREACH OF BARS
        for index, row in tqdm(filteredDf.iterrows()):
            if index > len(filteredDf)-max(barsAfter)-1:
                 continue

            # Current bar has closed outside upper band
            if row.RSI > threshUp and row.close > row.topBand:
    #            print(pd.to_datetime(row.Date, unit='s'))

                for n in barsAfter:
                    nextBars = filteredDf.iloc[index+1:index+n+1]

                    if min(nextBars.low) < (row.close - pips):
                        resultDict[key][n].append(True)
                    else:
                        resultDict[key][n].append(False)

            elif row.RSI < threshDown and row.close < row.bottomBand:

                for n in barsAfter:
                    nextBars = filteredDf.iloc[index+1:index+n+1]

                    if max(nextBars.high) > (row.close + pips):
                        resultDict[key][n].append(True)
                    else:
                        resultDict[key][n].append(False)
                        
    for key, val in resultDict.items():
        for i, j in val.items():
            n, c = np.unique(j, return_counts=True)
            print(key, i, n, c, c[1]/sum(c)*100)
        print('\n')

GBPUSD




15 3 [False  True] [ 324 1029] 76.05321507760532
15 5 [False  True] [ 248 1105] 81.67036215816704
15 12 [False  True] [ 161 1192] 88.10051736881005


30 3 [False  True] [149 640] 81.11533586818757
30 5 [False  True] [116 673] 85.29784537389101
30 12 [False  True] [ 75 714] 90.49429657794677


60 3 [False  True] [ 63 395] 86.24454148471615
60 5 [False  True] [ 53 405] 88.42794759825327
60 12 [False  True] [ 43 415] 90.61135371179039


GBPCHF




15 3 [False  True] [313 877] 73.69747899159664
15 5 [False  True] [251 939] 78.90756302521008
15 12 [False  True] [ 167 1023] 85.96638655462185


30 3 [False  True] [133 563] 80.89080459770115
30 5 [False  True] [103 593] 85.20114942528735
30 12 [False  True] [ 72 624] 89.65517241379311


60 3 [False  True] [ 45 333] 88.09523809523809
60 5 [False  True] [ 38 340] 89.94708994708994
60 12 [False  True] [ 30 348] 92.06349206349206


GBPAUD




15 3 [False  True] [229 845] 78.67783985102422
15 5 [False  True] [179 895] 83.33333333333334
15 12 [False  True] [120 954] 88.8268156424581


30 3 [False  True] [ 88 486] 84.66898954703834
30 5 [False  True] [ 67 507] 88.32752613240417
30 12 [False  True] [ 42 532] 92.6829268292683


60 3 [False  True] [ 31 279] 90.0
60 5 [False  True] [ 21 289] 93.2258064516129
60 12 [False  True] [ 15 295] 95.16129032258065


GBPCAD




15 3 [False  True] [ 259 1022] 79.78142076502732
15 5 [False  True] [ 205 1076] 83.9968774395004
15 12 [False  True] [ 130 1151] 89.85167837626854


30 3 [False  True] [ 97 592] 85.92162554426706
30 5 [False  True] [ 77 612] 88.82438316400581
30 12 [False  True] [ 54 635] 92.16255442670537


60 3 [False  True] [ 40 314] 88.70056497175142
60 5 [False  True] [ 33 321] 90.67796610169492
60 12 [False  True] [ 29 325] 91.80790960451978


GBPNZD




15 3 [False  True] [175 794] 81.94014447884416
15 5 [False  True] [140 829] 85.55211558307533
15 12 [False  True] [ 98 871] 89.88648090815273


30 3 [False  True] [ 70 468] 86.98884758364312
30 5 [False  True] [ 52 486] 90.33457249070632
30 12 [False  True] [ 34 504] 93.68029739776952


60 3 [False  True] [ 25 234] 90.34749034749035
60 5 [False  True] [ 20 239] 92.27799227799228
60 12 [False  True] [ 12 247] 95.36679536679536


